In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ddm
import os 
import csv
import helper
from model_definitions import ModelTtaBounds

In [2]:
for folder in ['cross_validation_9', 'cross_validation_8', 'cross_validation_4']:
    helper.merge_csv('../model_fit_results/%s' % (folder))

../model_fit_results/cross_validation_9\subj_129.csv
../model_fit_results/cross_validation_9\subj_389.csv
../model_fit_results/cross_validation_9\subj_525.csv
../model_fit_results/cross_validation_9\subj_616.csv
../model_fit_results/cross_validation_9\subj_618.csv
../model_fit_results/cross_validation_9\subj_642.csv
../model_fit_results/cross_validation_9\subj_755.csv
../model_fit_results/cross_validation_9\subj_all.csv
../model_fit_results/cross_validation_8\subj_129.csv
../model_fit_results/cross_validation_8\subj_389.csv
../model_fit_results/cross_validation_8\subj_525.csv
../model_fit_results/cross_validation_8\subj_616.csv
../model_fit_results/cross_validation_8\subj_618.csv
../model_fit_results/cross_validation_8\subj_642.csv
../model_fit_results/cross_validation_8\subj_755.csv
../model_fit_results/cross_validation_8\subj_all.csv
../model_fit_results/cross_validation_4\subj_129.csv
../model_fit_results/cross_validation_4\subj_389.csv
../model_fit_results/cross_validation_4\subj_5

In [3]:
def get_model_measures(model, condition):
    sol = model.solve(condition)
    return(condition['tta_condition'], condition['d_condition'], sol.prob_correct(), sol.mean_decision_time())

def get_model_rt_distr(model, condition, kind='cdf'):
    sol = model.solve(condition)
    return(pd.DataFrame({'tta_condition': condition['tta_condition'],
                         'd_condition':  condition['d_condition'],
                         't': model.t_domain(),
                         'rt_corr_distr': (sol.cdf_corr() if kind=='cdf' else sol.pdf_corr())/sol.prob_correct()}))

def initialize_model(param_set):
    overlay = ModelTtaBounds.OverlayNonDecisionGaussian(nondectime=param_set.ndt_location, ndsigma=param_set.ndt_scale)
    model = ddm.Model(name='TTA- and d-dependent drift and bounds and random nondecision time',
                         drift=ModelTtaBounds.DriftTtaDistance(alpha=param_set.alpha, 
                                                               beta=param_set.beta, theta=param_set.theta),
                         noise=ddm.NoiseConstant(noise=param_set.noise),
                         bound=ModelTtaBounds.BoundCollapsingTta(b_0=param_set.b_0, 
                                                                 k=param_set.k, tta_crit=param_set.tta_crit),
                         overlay=overlay, T_dur=ModelTtaBounds.T_dur)
    return(model)

def simulate_model(param_set, conditions, ret='measures'): 
    '''
    Set ret to 'measures' or 'rt_cdf' or 'rt_pdf' for saving p_turn and mean RT or RT CDF or RT PDF
    '''
    model = initialize_model(param_set)
    if ret=='measures':
        sim_result = pd.DataFrame([get_model_measures(model, condition) for condition in conditions],
                                  columns=['tta_condition', 'd_condition', 'is_turn_decision', 'RT'])
    else:
        sim_result = pd.concat([get_model_rt_distr(model, condition, kind=ret[-3:]) for condition in conditions])
    sim_result['subj_id'] = param_set.subj_id
    return sim_result

In [8]:
def save_sim_results(file_name, conditions=None, cross_validation=False, ret='measures'):
    path = '../model_fit_results/'
    file_path = os.path.join(path, file_name)
    parameters = pd.read_csv(file_path).rename(columns={'tta': 'tta_condition', 'd': 'd_condition'})
    
    if ret=='measures':
        if cross_validation:
            sim_results = [simulate_model(param_set, [param_set[['tta_condition', 'd_condition']].to_dict()]) 
                               for idx, param_set in parameters.iterrows()]
        else:
            sim_results = [simulate_model(param_set, conditions, ret='measures') 
                                  for idx, param_set in parameters.iterrows()]    
        sim_results = pd.concat(sim_results)
        sim_results.to_csv(file_path.replace('parameters_fitted', 'measures'), index=False)
    else:
        sim_results = [simulate_model(param_set, conditions, ret=ret) for idx, param_set in parameters.iterrows()]
        sim_results = pd.concat(sim_results)
        sim_results.to_csv(file_path.replace('parameters_fitted', ret), index=False)
#     return sim_results

# Simulate model trained on full data

In [9]:
conditions = [{'tta_condition': tta, 'd_condition': d} 
                       for tta in np.linspace(2,8,13)
                       for d in np.linspace(60,180,13)]

In [10]:
save_sim_results('full_data_parameters_fitted.csv', conditions, cross_validation=False, ret='measures')

In [12]:
save_sim_results('full_data_parameters_fitted.csv', conditions, cross_validation=False, ret='rt_cdf')

# Simulating cross-validated model

In [13]:
conditions = [{'tta_condition': tta, 'd_condition': d} 
                       for tta in [4,5,6]
                       for d in [90,120,150]]

In [14]:
save_sim_results('cross_validation_8_parameters_fitted.csv', conditions, ret='measures')

In [15]:
save_sim_results('cross_validation_4_parameters_fitted.csv', conditions, ret='measures')